# Final project IR-WA

Èlia Armengol 205744 Gabriela Cid 206085

In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import time
import math
import collections
from collections import Counter
from collections import defaultdict
from wordcloud import WordCloud, ImageColorGenerator
from config import *
import nltk
import pandas as pd
import seaborn as sns
import os
import numpy as np
import matplotlib.pyplot as plt
import nltk
import string
import unidecode
import re
from array import array
from numpy import linalg as la
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import preprocess_string
# Set Pandas to show all the columns
pd.set_option('display.max_columns', None)

In [2]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor
import json
import datetime
import re
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words("english"))
from nltk.stem import PorterStemmer
porter = PorterStemmer()

## 1 - Data collection

We choose to recollect information from the tweets with the theme Black Lives Matter

In [3]:
# download tweepy

## access token informations 
access_token1 = "2801892334-ahdScaxkY1zLMOX4WpKJRHin470YU0RG7cA7Zxo"
access_token_secret1 = "gktWBlIyJYDaN0R3ST283iYd9coDbZPT505nSZHcmlLmN"

consumer_key1 = "JdxyZOEGkF6IHW5wykjSNd8or"
consumer_secret1 = "43cHQOySq5bzWXV3X7HNjdBi4lSQbi37UOgEAnnj2U2AHttL8R"
bearer = "AAAAAAAAAAAAAAAAAAAAABq5KQEAAAAAmxWRR2sLSoQ%2BKL2L14R4fEDySVI%3Dy1o9dfDqaPzi70HSpYTZjMEoaaPOMjxVWxtXpulZ5z8j1buVr8"

In [4]:
auth = OAuthHandler(consumer_key1, consumer_secret1)
auth.set_access_token(access_token1, access_token_secret1)
api = API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [5]:
class MyStreamListener(StreamListener):
    """
    Twitter listener, collects streaming tweets and output to a file
    """
    def __init__(self, api, OUTPUT_FILENAME, stop_condition=10):
        """
        initialize the stream, with num. of tweets and saving the outputfile
        """
        
        # this line is needed to import the characteristics of the streaming API
        super(MyStreamListener, self).__init__()
        
        # to-count the number of tweets collected        
        self.num_tweets = 0
        
        # save filename
        self.filename = OUTPUT_FILENAME
        
        # stop-condition
        self.stop_condition = stop_condition
        

    def on_status(self, status):
        
        """
        this function runs each time a new bunch of tweets is retrived from the streaming
        """
        
        with open(self.filename, "a+") as f:
            tweet = status._json
            
            f.write(json.dumps(tweet) + '\n')
            #self.output.append(tweet)
            self.num_tweets += 1
        
            # Stop condition        
            if self.num_tweets <= self.stop_condition:
                return True
            else:
                return False
        

    def on_error(self, status):
        """
        function useful to handle errors. It's possible to personalize it 
        depending on the way we want to handle errors
        """
        
        print(status)
        #returning False in on_error disconnects the stream
        return False

In [ ]:
%%time

OUTPUT_FILENAME = "data/one.json"
stop_condition = 1500

l = MyStreamListener(api, OUTPUT_FILENAME, stop_condition)
# here we recall the Stream Class from Tweepy to input the authentication info and our personalized listener 
stream = Stream(auth=api.auth, listener=l)


# keywords we may want decide to track 
TRACKING_KEYWORDS = ["#BLM", "#BlackLivesMatter", "#BreonnaTaylor", "#GeorgeFloyd", "black lives", "black", "black people"]
stream.filter(
    track=TRACKING_KEYWORDS, 
    is_async=False, 
    languages = ["en"]
)

## 2 - Build the Search Engine

In [ ]:
with open("data/one.json", "rb") as f:
    data = f.readlines()
    data = [json.loads(str_) for str_ in data]

df_tweets = pd.DataFrame.from_records(data)

df_tweets.head()

In [ ]:
df_tweets = df_tweets[df_tweets['extended_tweet'].notna()]
df_tweets = df_tweets.reset_index(drop=True)
df_tweets = df_tweets.drop(['id', 'id_str'], axis=1)
df_tweets["index"] = df_tweets.index


In [ ]:
lines=[]
for i in range(0,len(df_tweets)):
    has= []
    for j in range(0,len(df_tweets['extended_tweet'].values[i]['entities']['hashtags'])):
        has.append(df_tweets['extended_tweet'].values[i]['entities']['hashtags'][j]['text'])
        
    lines.append(str(df_tweets["index"].values[i]) +'|'+
        str(df_tweets['extended_tweet'].values[i]['full_text'])+'|'+
                  str(df_tweets['user'].values[i]['name']) +'|'+ 
                    str(df_tweets['created_at'].values[i]) +'|'+ str(has) +'|'
                + str(df_tweets['favorite_count'].values[i])+'|'+str(df_tweets['retweet_count'].values[i])
                 +'|'+ str(df_tweets['source'].values[i]))


Create the inverted index

In [ ]:
def listToString(s):  
    
    # initialize an empty string 
    str1 = ""  
    
    # traverse in the string   
    for ele in s:  
        str1 += ele   
    
    # return string   
    return str1  

In [ ]:
def remove_accents(text):
    if text:
        # TODO: remove the accents from text (tip: search for unicode)
        return unidecode.unidecode(text)
        
    # In case there is no text
    return ""

def remove_punctuation_marks(text):
    if text:
        # TODO: Remove the punctuation marks from text (tip: search for translate and maketrans)
        return text.translate(str.maketrans('', '', string.punctuation))
    # In case there is no text
    return ""

def text_to_lower_case(text):
    if text:
        # TODO: convert text to lower case
        return text.lower()
    
    # In case there is no text
    return ""

def remove_emojis(text):
    if text:
        # TODO: Remove emojis (tip: search for encode - decode)
        return text.encode('ascii','ignore').decode('ascii')

    # In case there is no text
    return ""

def remove_multiple_whitespaces(text):
    if text:
        # TODO: remove multiple whitespaces (tip: search for regex and replace)
        return re.sub("\s\s+", " ", text)
    # In case there is no text
    return ""



def remove_text_marks(text):
    if text:
        
        #remove @+usernames
        text =  " ".join(filter(lambda x:x[0]!='@', text.split()))
            
        # TODO: replace characters like it\'s by its
        text = re.sub(r"\' ", " ",text)
        
        # TODO: replace *, ?, ... by spaces
        text = re.sub(r'[^\w]', " ",text)
        
        return text.strip()
    
    # In case there is no text
    return ""

def split_text_and_numbers(text):
    return text

def remove_alone_numbers(text):
    if text:
        # TODO: keep only text
        text = re.sub(r"\d", "",text)
        
        return text
    
    return ""

def clean_text(text):
    # Apply the different functions in order to clean the text
    text = text_to_lower_case(text)
    text = remove_text_marks(text)
    text = remove_punctuation_marks(text)
    text = remove_accents(text)
    text = remove_emojis(text)
    text = split_text_and_numbers(text)
    text = remove_alone_numbers(text)
    text = remove_multiple_whitespaces(text)
    
    # Return
    return text

In [ ]:
def getTerms(line):
    """
    Preprocess the text removing stop words, stemming,
    transforming in lowercase and return the tokens of the text.
    
    Argument:
    line -- string (text) to be preprocessed
    
    Returns:
    line - a list of tokens corresponding to the input text after the preprocessing
    """
        
    stemming = PorterStemmer()
    stops = set(stopwords.words("english"))
    ## START CODE
    line = clean_text(line)
    line= line.split(" ") ## Tokenize the text to get a list of terms
    
    line=[x for x in line if x not in stops]  
    
    
    ##eliminate the stopwords (HINT: use List Comprehension)
    
    line=[stemming.stem(word) for word in line] ## perform stemming (HINT: use List Comprehension)
    
    return line

In [ ]:
def create_index(lines):
    """
    Impleent the inverted index
    
    Argument:
    lines -- collection of Wikipedia articles
    
    Returns:
    index - the inverted index (implemented through a python dictionary) containing terms as keys and the corresponding 
    list of document these keys appears in (and the positions) as values.
    """
    index=defaultdict(list) 
    titleIndex = {} # dictionary to map page titles to page ids
    for line in lines: # Remember, lines contain all documents, each line is a document
        line_arr = line.split("|")
        page_id = int(line_arr[0] )
        terms = getTerms(''.join(line_arr[1:])) #page_title + page_text
        title = line_arr[1]            
        titleIndex[page_id]=title  ## we do not need to apply get terms to title because it used only to print titles and not in the index
        
        ## ===============================================================        
        ## create the index for the current doc and store it in termdictPage
        ## termdictPage ==> { ‘term1’: [currentdoc, [list of positions]], ...,‘termn’: [currentdoc, [list of positions]]}
        
        ## Example: if the curr_doc has id 1 and his text is 
        ## "web retrieval information retrieval":
        
        ## termdictPage ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,4]], ‘information’: [1, [2]]}
        
        ## the term ‘web’ appears in document 1 in positions 0, 
        ## the term ‘retrieval’ appears in document 1 in positions 1 and 4
        ## ===============================================================
        
        termdictPage={}

        for position, term in enumerate(terms): # terms contains page_title + page_text. Loop over all terms
            try:
                # if the term is already in the index for the current page (termdictPage)
                # append the position to the corrisponding list
                
        ## START CODE
                termdictPage[term][1].append(position)  
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                termdictPage[term]=[page_id, array('I',[position])] #'I' indicates unsigned int (int in python)
            
        #merge the current page index with the main index
        for termpage, postingpage in termdictPage.items():
            index[termpage].append(postingpage)
        
        ## END CODE                    
                    
    return index, titleIndex

In [ ]:
index, titleIndex = create_index(lines)

In [ ]:
print("Index results for the term 'stop': {}\n".format(index['stop']))

In [ ]:
bag_of_words = {}
for i in titleIndex:
    # remove "RT" string indicating a retweet
    text = titleIndex[i].replace("RT", "").strip()
    
    # lowering text
    text = text.lower()
    
    # removing all the punctuations
    text = re.sub(r'[^\w\s]','',text).strip()
    
    # tokenize the text
    lst_text = text.split()
    
    # remove stopwords
    lst_text = [x for x in lst_text if x not in STOPWORDS]
    
        
    # create bag-of-words - for each word the frequency of the word in the corpus
    for w in lst_text:
        if w not in bag_of_words:
            bag_of_words[w] = 0
        bag_of_words[w] += 1

In [ ]:
def plot_wordcloud(title, dic_):
    fig, ax = plt.subplots(1, 1, figsize=(18,7))
    wordcloud = WordCloud(background_color="white",width=1600, height=800)
    wordcloud = wordcloud.generate_from_frequencies(dic_)
    ax.axis("off")     
    ax.imshow(wordcloud, interpolation='bilinear')

    ax.set_title(title)
    plt.tight_layout()
    fig.subplots_adjust(top=0.8)
    plt.show()

In [ ]:
plot_wordcloud("WordCloud - All Tweets", bag_of_words)

In [ ]:
bag_of_words = dict(sorted(bag_of_words.items(), key=lambda item: item[1]))
most_common_words= []
values = []
for i in range(1, 10):
    most_common_words.append(list(bag_of_words)[-i])
    values.append(bag_of_words[list(bag_of_words)[-i]])
#most_common_words
plt.figure(figsize = (10,10))
plt.bar(most_common_words,values)


# Inverted Index

In [ ]:
def create_index_tfidf(lines, numDocuments):
    """
    Implement the inverted index and compute tf, df and idf
    
    Argument:
    lines -- collection of Wikipedia articles
    numDocuments -- total number of documents
    
    Returns:
    index - the inverted index (implemented through a python dictionary) containing terms as keys and the corresponding 
    list of document these keys appears in (and the positions) as values.
    tf - normalized term frequency for each term in each document
    df - number of documents each term appear in
    idf - inverse document frequency of each term
    """
        
    index=defaultdict(list)
    tf=defaultdict(list) #term frequencies of terms in documents (documents in the same order as in the main index)
    df=defaultdict(int)         #document frequencies of terms in the corpus
    titleIndex=defaultdict(str)
    idf=defaultdict(float)
    
    for line in lines:
        line_arr = line.split("|")
        page_id = int(line_arr[0] )        
        terms = getTerms(''.join(line_arr[1:])) #page_title + page_text
        title = line_arr[1]            
        titleIndex[page_id]=title          
        
        ## ===============================================================        
        ## create the index for the **current page** and store it in termdictPage
        ## termdictPage ==> { ‘term1’: [currentdoc, [list of positions]], ...,‘termn’: [currentdoc, [list of positions]]}
        
        ## Example: if the curr_doc has id 1 and his text is 
        ## "web retrieval information retrieval":
        
        ## termdictPage ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,4]], ‘information’: [1, [2]]}
        
        ## the term ‘web’ appears in document 1 in positions 0, 
        ## the term ‘retrieval’ appears in document 1 in positions 1 and 4
        ## ===============================================================

       
        termdictPage={}

        for position, term in enumerate(terms): ## terms contains page_title + page_text
            try:
                # if the term is already in the dict append the position to the corrisponding list
                termdictPage[term][1].append(position) 
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                termdictPage[term]=[page_id, array('I',[position])] #'I' indicates unsigned int (int in python)
                
        #normalize term frequencies
        # Compute the denominator to normalize term frequencies (formula 2 above)
        # norm is the same for all terms of a document.
        norm=0
        for term, posting in termdictPage.items(): 
            # posting is a list containing doc_id and the list of positions for current term in current document: 
            # posting ==> [currentdoc, [list of positions]] 
            # you can use it to inferr the frequency of current term.
            norm+=len(posting[1])**2
        norm=math.sqrt(norm)


        #calculate the tf(dividing the term frequency by the above computed norm) and df weights
        for term, posting in termdictPage.items():     
            # append the tf for current term (tf = term frequency in current doc/norm)
            tf[term].append(np.round(len(posting[1])/norm,4))  ## SEE formula (1) above
            #increment the document frequency of current term (number of documents containing the current term)
            df[term]+= 1  # increment df for current term
        
        #merge the current page index with the main index
        for termpage, postingpage in termdictPage.items():
            index[termpage].append(postingpage)
            
        # Compute idf following the formula (3) above. HINT: use np.log
        for term in df:
            idf[term] = np.round(np.log(float(numDocuments/df[term])),4)
            
    return index, tf, df, idf, titleIndex


In [ ]:
start_time = time.time()
numDocuments = len(lines)
index, tf, df, idf, titleIndex = create_index_tfidf(lines, numDocuments)
print("Total time to create the index: {} seconds" .format(np.round(time.time() - start_time,2)))

In [ ]:
print("Index results for the term 'stop': {}\n".format(index['stop']))

# Ranked documents

### TF-IDF + cosine similarity

In [ ]:

def rankDocuments(terms, docs, index, idf, tf, titleIndex):
    """
    Perform the ranking of the results of a search based on the tf-idf weights
    
    Argument:
    terms -- list of query terms
    docs -- list of documents, to rank, matching the query
    index -- inverted index data structure
    idf -- inverted document frequencies
    tf -- term frequencies
    titleIndex -- mapping between page id and page title
    
    Returns:
    Print the list of ranked documents
    """
        
    # I'm interested only on the element of the docVector corresponding to the query terms 
    # The remaing elements would became 0 when multiplied to the queryVector
    docVectors=defaultdict(lambda: [0]*len(terms)) # I call docVectors[k] for a nonexistent key k, the key-value pair (k,[0]*len(terms)) will be automatically added to the dictionary
    queryVector=[0]*len(terms)    

    # compute the norm for the query tf
    query_terms_count = collections.Counter(terms) # get the frequency of each term in the query. 
    # Example: collections.Counter(["hello","hello","world"]) --> Counter({'hello': 2, 'world': 1})
    # HINT: use when computing tf for queryVector
    
    query_norm = la.norm(list(query_terms_count.values()))
    
    for termIndex, term in enumerate(terms): #termIndex is the index of the term in the query
        if term not in index:
            continue
                    
        ## Compute tf*idf(normalize tf as done with documents)
        queryVector[termIndex]=query_terms_count[term]/query_norm * idf[term] 
        
        # Generate docVectors for matching docs
        for docIndex, (doc, postings) in enumerate(index[term]):
            # Example of [docIndex, (doc, postings)]
            # 0 (26, array('I', [1, 4, 12, 15, 22, 28, 32, 43, 51, 68, 333, 337]))
            # 1 (33, array('I', [26, 33, 57, 71, 87, 104, 109]))
            # term is in doc 26 in positions 1,4, .....
            # term is in doc 33 in positions 26,33, .....
            
            #tf[term][0] will contain the tf of the term "term" in the doc 26            
            if doc in docs:
                docVectors[doc][termIndex]=tf[term][docIndex] * idf[term]  # TODO: check if multiply for idf

    # calculate the score of each doc
    # compute the cosine similarity between queyVector and each docVector:
    # HINT: you can use the dot product because in case of normalized vectors it corresponds to the cosine siilarity
    # see np.dot
    
    docScores=[ [np.dot(curDocVec, queryVector), doc] for doc, curDocVec in docVectors.items() ]
    docScores.sort(reverse=True)
    resultDocs=[x[1] for x in docScores]
    #print document titles instead if document id's
    #resultDocs=[ titleIndex[x] for x in resultDocs ]
    if len(resultDocs) == 0:
        print("No results found, try again")
        query = input()
        docs = search_tf_idf(query, index)    
    #print ('\n'.join(resultDocs), '\n')
    return resultDocs

In [ ]:
def search_tf_idf(query, index):
    '''
    output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    '''
    query=getTerms(query)
    docs=set()
    for term in query:
        try:
            # store in termDocs the ids of the docs that contain "term"                        
            termDocs=[posting[0] for posting in index[term]]
            
            # docs = docs Union termDocs
            docs |= set(termDocs)
        except:
            #term is not in index
            pass
    docs=list(docs)
    ranked_docs = rankDocuments(query, docs, index, idf, tf, titleIndex)   
    return ranked_docs

In [ ]:
print("Insert your query:\n")
query = input()
ranked_docs = search_tf_idf(query, index)    
top = 20

print("\n======================\nTop {} results out of {} for the seached query:\n".format(top, len(ranked_docs)))
for d_id in ranked_docs[:top] :
    print("page_id= {} - page_title: {}".format(d_id, titleIndex[d_id]))

### Our score

In [ ]:
def search(query, index):
    '''
    The output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    '''
    query=getTerms(query)
    docs=set()
    for term in query:
    ## START DODE
        try:
            # store in termDocs the ids of the docs that contain "term"                        
            termDocs=[posting[0] for posting in index[term]]
            # docs = docs Union termDocs
            docs |= set(termDocs)
        except:
            #term is not in index
            pass
    docs=list(docs)
    return docs

In [ ]:
print("Insert your query:\n")
query = input()
docs = search(query, index)    
top = 20

print("\n======================\nSample of {} results out of {} for the seached query:\n".format(top, len(docs)))
for d_id in docs[:top] :
    print("page_id= {} - page_title: {}".format(d_id, titleIndex[d_id]))